In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\shasw\anaconda3\envs\MediBotics\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#Extract data from pdf
def load_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    doc=loader.load()
    return doc

In [5]:
data_from_pdf=load_file(data='Data/')

In [8]:
def process_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=300)
    texts = text_splitter.split_documents(documents)
    return texts


In [1]:
chunks=process_documents(data_from_pdf)

NameError: name 'process_documents' is not defined

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

In [11]:
def download_hugging_embeddings():
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

In [12]:
embedding = download_hugging_embeddings()

In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
PINECONE_API_KEY = os.getenv('Pinecone_API_Key')


In [15]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc=Pinecone(api_key=PINECONE_API_KEY)

index_name="medibotics"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",
                            region="us-east-1"
            )
    )

In [16]:
import os
os.environ["Pinecone_API_Key"]=PINECONE_API_KEY

In [17]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=chunks,
    index_name=index_name,
    embedding=embedding
)


c:\Users\shasw\anaconda3\envs\MediBotics\lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [18]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)


In [32]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":2})

In [33]:
retrieved_docs=retriever.invoke("What is diabetes?")

In [34]:
GOOGLE_2_0_FLASH_API_KEY = os.getenv('GOOGLE_API_KEY')

In [35]:
import os
os.environ["GOOGLE_2_0_FLASH_API_KEY"]="GOOGLE_API_KEY"

In [38]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI( model="gemini-1.5-flash",temperature=0,max_output_tokens=512)

In [39]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrived context to answer"
    "the questions. If you don't know the answer , say that you"
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt= ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    ("user","{input}"),
])

In [40]:
question_answer=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer)

In [43]:
response=rag_chain.invoke({"input":"What is Stats?"})
print(response["answer"])

I'm sorry, but I cannot answer this question.  The provided text focuses on intelligence testing and Wegener's granulomatosis; it does not define or explain statistics.
